# Gathering all the data, then doing a clean for a master dataset.

In [ ]:
# Import the needed libraries
import numpy as np
import pandas as pd
from datetime import datetime as dt
import itertools


In [ ]:
# Load CSV data sets into a dataframe

loc = "/mnt/c/Users/anton/Documents/Projects/Predicting_WaterlooHealthInspections/datasets/"

raw_dataset_0 = pd.read_csv(loc + 'Facilities_OpenData.csv')
raw_dataset_1 = pd.read_csv(loc + 'Infractions_OpenData.csv')
raw_dataset_2 = pd.read_csv(loc + 'Inspections_OpenData.csv')

In [ ]:
# Parse data as time 

def parse_date1(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d').date()

# two different date formats present
def parse_date2(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y/%m/%d').date()


raw_dataset_1.InspectionDate = raw_dataset_1.InspectionDate.apply(parse_date1)
raw_dataset_2.INSPECTION_DATE = raw_dataset_2.INSPECTION_DATE.apply(parse_date2)
